In [1]:
import json
import pprint
import numpy as np
import pandas as pd
import seaborn as sns
import geopy.distance
from shapely.wkb import loads
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2

/var/folders/0r/13xz5g6j45589319bn94k47w0000gn/T/ipykernel_45424/3438936138.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
algo_df = pd.read_csv("./allocation_algorithm_logs_20240105.csv")
algo_df

,request,AVAILABLE_INVENTORY,PLAN
0,"{\n ""destination"": {\n ""branch_number"": """"...","[\n {\n ""BranchNumber"": ""060293"",\n ""Cr...","{\n ""allocation_id"": ""018cd7ee-fab0-77d9-b417..."
1,"{\n ""destination"": {\n ""branch_number"": """"...","[\n {\n ""BranchNumber"": ""330302"",\n ""Cr...","{\n ""allocation_id"": ""018cd951-c168-7149-8434..."
2,"{\n ""destination"": {\n ""branch_number"": ""0...","[\n {\n ""BranchNumber"": ""000025"",\n ""Cr...","{\n ""allocation_id"": ""018cdb42-2356-7261-8393..."
3,"{\n ""destination"": {\n ""branch_number"": ""0...","[\n {\n ""BranchNumber"": ""060940"",\n ""Cr...","{\n ""allocation_id"": ""018cda6a-5617-76ff-965b..."
4,"{\n ""destination"": {\n ""branch_number"": """"...","[\n {\n ""BranchNumber"": ""020400"",\n ""Cr...","{\n ""allocation_id"": ""018cd939-895f-7fea-a0e3..."
...,...,...,...
3805,"{\n ""destination"": {\n ""branch_number"": ""0...","[\n {\n ""BranchNumber"": ""000337"",\n ""Cr...","{\n ""allocation_id"": ""018cd984-1386-711e-bd3b..."
3806,"{\n ""destination"": {\n ""branch_number"": ""0...","[\n {\n ""BranchNumber"": ""060948"",\n ""Cr...","{\n ""allocation_id"": ""018cd94b-7815-7de9-a94c..."
3807,"{\n ""destination"": {\n ""branch_number"": """"...","[\n {\n ""BranchNumber"": ""016347"",\n ""Cr...","{\n ""allocation_id"": ""018cd8f1-d26a-7953-95ec..."
3808,"{\n ""destination"": {\n ""branch_number"": """"...","[\n {\n ""BranchNumber"": ""020466"",\n ""Cr...","{\n ""allocation_id"": ""018cd935-f75e-7960-9aca..."


In [3]:
algo_df.dtypes

request                object
AVAILABLE_INVENTORY    object
PLAN                   object
dtype: object

In [4]:
algo_df.isna().sum()

request                0
AVAILABLE_INVENTORY    0
PLAN                   0
dtype: int64

In [5]:
algo_df.isnull().sum()

request                0
AVAILABLE_INVENTORY    0
PLAN                   0
dtype: int64

In [6]:
algo_df.duplicated().sum()

0

In [7]:
algo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   request              3810 non-null   object
 1   AVAILABLE_INVENTORY  3810 non-null   object
 2   PLAN                 3810 non-null   object
dtypes: object(3)
memory usage: 89.4+ KB


In [8]:
algo_df.describe()

,request,AVAILABLE_INVENTORY,PLAN
count,3810,3810,3810
unique,3810,3805,3810
top,"{\n ""destination"": {\n ""branch_number"": """"...","[\n {\n ""BranchNumber"": ""060773"",\n ""Cr...","{\n ""allocation_id"": ""018cd7ee-fab0-77d9-b417..."
freq,1,3,1


In [9]:
# sample = algo_df.iloc[5]
# sample
dt = []
print(algo_df.shape)
sample = None
for instance in algo_df.values:
  instance
  a = json.loads(instance[0])
  if len(a['items']) > 0: # get a sample with many items to know more, 
    sample = instance
    break
  # if len( dt) > 3: break # stop loop after getting 3 instance of delivery door to door
  # if a['destination']['delivery_type'] == 'DoorToDoor': dt.append(instance)
sample

(3810, 3)


array(['{\n  "destination": {\n    "branch_number": "",\n    "delivery_type": "DoorToDoor",\n    "latitude": -34.0331987,\n    "longitude": 18.4902276,\n    "postal_code": "7880",\n    "province": "Western Cape"\n  },\n  "dry_run": false,\n  "external_order_reference": "B4403050-01",\n  "items": [\n    {\n      "alternative_sku": null,\n      "alternative_trading_company": null,\n      "excluded_branches": null,\n      "order_item_id": "cb47a624-f2c6-5819-8a41-60685135920d",\n      "previous_reservation_ids": null,\n      "quantity": 1,\n      "sku": "60107365",\n      "trading_company_number": "06"\n    }\n  ],\n  "order_created_at": "2024-01-05T04:43:30.778689Z",\n  "order_id": "b3ef66cf-1ee7-5a38-b607-049a1d151e0a"\n}',
       '[\n  {\n    "BranchNumber": "060293",\n    "CreatedAt": "2023-10-12T07:55:06.692344Z",\n    "DeletedAt": null,\n    "FacilityActive": true,\n    "FacilityAtCapacity": false,\n    "FacilityCourierCActive": false,\n    "FacilityBuffer": 1,\n    "FacilityCoordin

In [10]:
# print(dt[0][0]),
# print(dt[0][1]),
# print(dt[0][2])

In [11]:
request = json.loads(sample[0])
stocks = json.loads(sample[1])
plan = json.loads(sample[2])

In [12]:
print(request.keys())
print("number items: ", len(request['items']))
# 
request

dict_keys(['destination', 'dry_run', 'external_order_reference', 'items', 'order_created_at', 'order_id'])
number items:  1


{'destination': {'branch_number': '',
  'delivery_type': 'DoorToDoor',
  'latitude': -34.0331987,
  'longitude': 18.4902276,
  'postal_code': '7880',
  'province': 'Western Cape'},
 'dry_run': False,
 'external_order_reference': 'B4403050-01',
 'items': [{'alternative_sku': None,
   'alternative_trading_company': None,
   'excluded_branches': None,
   'order_item_id': 'cb47a624-f2c6-5819-8a41-60685135920d',
   'previous_reservation_ids': None,
   'quantity': 1,
   'sku': '60107365',
   'trading_company_number': '06'}],
 'order_created_at': '2024-01-05T04:43:30.778689Z',
 'order_id': 'b3ef66cf-1ee7-5a38-b607-049a1d151e0a'}

In [13]:
print(len(stocks))
[ stock for stock in stocks if stock['BranchNumber'] == '060905']

14


[{'BranchNumber': '060905',
  'CreatedAt': '2023-10-12T07:54:52.341153Z',
  'DeletedAt': None,
  'FacilityActive': True,
  'FacilityAtCapacity': False,
  'FacilityCourierCActive': True,
  'FacilityBuffer': 1,
  'FacilityCoordinates': '0101000020E6100000960A2AAA7E993240897E6DFDF4F340C0',
  'FacilityID': '018a9335-7878-7fde-a9fd-57fc82d34963',
  'FacilityProvince': 'Western Cape',
  'FacilityRegion': 'LOCAL',
  'FacilityReservedQty': 25,
  'FacilitySKUActive': True,
  'FacilitySubType': None,
  'FacilityType': 'Store',
  'Id': '018b22e1-9a33-7364-8177-56b97367756f',
  'NormalisedSku': '06-60107365',
  'OnHandQuantity': 3,
  'QtyAvailable': 2,
  'Sku': '60107365',
  'StockSourceUpdatedAt': '2023-12-31T11:53:09Z',
  'TradingCompanyNumber': '06',
  'UpdatedAt': '2023-12-31T11:53:14.225731Z'}]

In [14]:
plan

{'allocation_id': '018cd7ee-fab0-77d9-b417-61fd0f96e865',
 'cancellations': None,
 'created_at': '2024-01-05T04:43:32.448771942Z',
 'dry_run': False,
 'external_order_reference': 'B4403050-01',
 'order_id': 'b3ef66cf-1ee7-5a38-b607-049a1d151e0a',
 'reservations': [{'alternative_sku': None,
   'alternative_trading_company': None,
   'branch_number': '060905',
   'courier': 'courier_c',
   'created_at': '2024-01-05T04:43:32.443897453Z',
   'excluded_branches': None,
   'omni_fulfilment_data': {'fulfilment_centre_code': 'STORE',
    'fulfilment_centre_description': 'Stores',
    'fulfilment_centre_id': 6},
   'order_item_id': 'cb47a624-f2c6-5819-8a41-60685135920d',
   'previous_reservation_ids': None,
   'quantity': 1,
   'reservation_id': '018cd7ee-fadb-77d9-acc0-cfc31a879d1c',
   'sku': '60107365',
   'trading_company_number': '06'}]}

# Testing Idea

In [24]:

def get_wkb_coords(wkb_string):
    """decode wkb-hex string... extract the coordinates... return results... """
    geom = loads(bytes.fromhex(wkb_string))
    coordinates = list(geom.coords)
    return coordinates

def _distance(lat1, lon1, lat2, lon2):
  coords_1 = (lat1, lon1)
  coords_2 = (lat2, lon2)
  return geopy.distance.geodesic(coords_1, coords_2).km


def haversine_distance(lat1, lon1, lat2, lon2):
    """convert lat and longitude to radians from degree... get difference between latitude and longitude... multiply value from haversine with earth radius in km ... return the distance in KM"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    radius_earth_km = 6371.0 # earth's radius based on google check

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = radius_earth_km * c
    return distance

In [16]:
request

{'destination': {'branch_number': '',
  'delivery_type': 'DoorToDoor',
  'latitude': -34.0331987,
  'longitude': 18.4902276,
  'postal_code': '7880',
  'province': 'Western Cape'},
 'dry_run': False,
 'external_order_reference': 'B4403050-01',
 'items': [{'alternative_sku': None,
   'alternative_trading_company': None,
   'excluded_branches': None,
   'order_item_id': 'cb47a624-f2c6-5819-8a41-60685135920d',
   'previous_reservation_ids': None,
   'quantity': 1,
   'sku': '60107365',
   'trading_company_number': '06'}],
 'order_created_at': '2024-01-05T04:43:30.778689Z',
 'order_id': 'b3ef66cf-1ee7-5a38-b607-049a1d151e0a'}

### Add Dc_type to the request

In [17]:
dc_types = pd.read_csv("./dc_branch_numbers.csv")
dc_types_values = { f'{row[0]}' :row[1] for row in dc_types.values }
len(dc_types_values.keys())

32

In [18]:
def add_dc_type_request(request_):
    """add a dc-type to the request for processing later"""
    if request_:
        delivery_type = request_["destination"]["delivery_type"]
        branch_number = request_["destination"]["branch_number"]
    # if the delivery type is [storepickup] but the branch number is not there, add it from dc_branch_number file
    if delivery_type.lower() == "storepickup" and branch_number:
        if branch_number in dc_types_values.keys():
            request_["dc_type"] = dc_types_values[branch_number]
    return request_
  
request = add_dc_type_request(request)
request

{'destination': {'branch_number': '',
  'delivery_type': 'DoorToDoor',
  'latitude': -34.0331987,
  'longitude': 18.4902276,
  'postal_code': '7880',
  'province': 'Western Cape'},
 'dry_run': False,
 'external_order_reference': 'B4403050-01',
 'items': [{'alternative_sku': None,
   'alternative_trading_company': None,
   'excluded_branches': None,
   'order_item_id': 'cb47a624-f2c6-5819-8a41-60685135920d',
   'previous_reservation_ids': None,
   'quantity': 1,
   'sku': '60107365',
   'trading_company_number': '06'}],
 'order_created_at': '2024-01-05T04:43:30.778689Z',
 'order_id': 'b3ef66cf-1ee7-5a38-b607-049a1d151e0a'}

### Get facilities with matched SKU

In [19]:

def _macthed_sku(stocks, request):
    """check if number items in request are more than 1. go over all the items. got over all stocks. check if sku matches and return ones where sku matches. make sure the facility is an active facility"""
    facilities = []
    # get all facilities with the item based on the sku
    if len(request["items"]) > 0:
        for request_item in request["items"]: 
            for stock in stocks:
                if request_item["sku"] == stock["Sku"] and stock['FacilityActive'] == True : # make sure facility is active and not closed
                    facilities.append(stock)
    return facilities

macthed_skus= _macthed_sku(stocks, request)
len(macthed_skus)

14

### Get facilities with available stocks

In [20]:
def _check_facility_quantity(macth_skus, request):
  """check the quantity of each item and return only those that have equal to or more quantity for those items"""
  have_quantity = []
  if len(request['items']) > 0 and len(macth_skus) > 0:
    for request_item in request['items']:
      for macth_sku in macth_skus:
        if request_item['quantity'] <= macth_sku['QtyAvailable']:
          have_quantity.append(macth_sku)  
  # 
  return have_quantity if len(have_quantity) > 0 else None

facility_check_quantity = _check_facility_quantity(macthed_skus, request)
len(facility_check_quantity)

14

### Add post codes to facilities

In [39]:
facility_post_codes_df = pd.read_csv("./bash_fcm_facilities.csv")
facility_post_codes_df["facility_id"].drop_duplicates(
    inplace=True
)  # remove duplicates facility IDs
print(facility_post_codes_df.shape)
facility_post_codes_values = {
    id: post_code for id, post_code in facility_post_codes_df.values
}
len(facility_post_codes_values.keys())

(5957, 2)


5947

In [41]:
def add_post_code_facility(filter_facilities):
    """"""
    facility_post_code = []
    for facility in filter_facilities:
        if facility["FacilityID"] in facility_post_codes_values.keys():
            facility["post_code"] = facility_post_codes_values[facility["FacilityID"]]
            facility_post_code.append(facility)
    #
    return facility_post_code

facility_post_code = add_post_code_facility(facility_check_quantity)
len(facility_post_code)

14

### Calculate distance from facility to destination, if coords are provided

In [42]:
def _get_distance_dest_facility(request, filtered_facilities ):
  """return all facilities that have coords and there is a distance between distination and facility"""
  distance_destination_facility = []
  # check if the distination has coords
  if len(request['items']) > 0 and request['destination']['latitude'] != None and request['destination']['longitude'] != None:
    request_latitude, request_longitude = request['destination']['latitude'], request['destination']['longitude']
    for facility in filtered_facilities:
      # check if the facility has hex-wkb coordinates
      if facility['FacilityCoordinates'] != None:
        facility_coords = get_wkb_coords(facility['FacilityCoordinates'])
        # calculate distance between distination and facility and store the distance in facility object
        if facility_coords != None:
          facility_latitude, facility_longitude = facility_coords[0]
          # distance_to_destination = _distance(request_latitude, request_longitude, facility_latitude, facility_longitude)
          distance_to_destination = haversine_distance(request_latitude, request_longitude, facility_latitude, facility_longitude)
          facility['distance'] = distance_to_destination
          distance_destination_facility.append(facility)
  # 
  return distance_destination_facility if len(distance_destination_facility) else None

facility_with_distance = _get_distance_dest_facility(request, facility_post_code)
len(facility_with_distance)
# facility_with_distance

14

### check if facility has courier_c in house and if can deliver based on distance constraint

In [23]:
def _facility_courier_c(filtered_facility):
  """get all facilities that can deliver with courier_c"""
  courier_c_facilities = []
  if len(filtered_facility) > 0:
    for facility in filtered_facility:
      if facility['FacilityCourierCActive'] == True and facility['distance'] <= 20:
        courier_c_facilities.append(facility)
  return courier_c_facilities if len(courier_c_facilities) > 0 else None

facility_courier_c_active = _facility_courier_c(facility_with_distance)
facility_courier_c_active